In [ ]:
import numpy as np
import pandas as pd

In [ ]:
sample = pd.read_csv('/kaggle/input/dls-data/sample_submission.csv.csv')
probs = np.load('/kaggle/input/1epoch-gemma-bf16/1epoch_gemma_2048_bf16.npy')
probs_new = np.load('/kaggle/input/1epoch-gemma-new-bf16/1epoch_gemma_new_bf16.npy')

In [ ]:
def f(row):
    res = []
    for i in range(len(row)):
        if row[i] == 1:
            res.append(i)
    return res

In [ ]:
tholds = []
# for thold_prep in range(5500, 2000, -1000):
for thold_prep in range(1000, 5501, 125):
    thold = thold_prep / 10000
    tholds.append(thold)
    
    pred = (probs > thold).astype(int)
    sample[f'target_{thold}'] = [f(row) for row in pred]
    sample[f'target_{thold}'] = sample[f'target_{thold}'].apply(lambda x: ' '.join(map(str, x)))
    
    pred_new = (probs_new > thold).astype(int)
    sample[f'target_new_{thold}'] = [f(row) for row in pred_new]
    sample[f'target_new_{thold}'] = sample[f'target_new_{thold}'].apply(lambda x: ' '.join(map(str, x)))

In [ ]:
for thold in tholds:
    sample[f'target_{thold}'] = sample[f'target_{thold}'].replace('', np.nan)
    sample[f'target_new_{thold}'] = sample[f'target_new_{thold}'].replace('', np.nan)

In [ ]:
for thold in reversed(tholds):
    print(f'curr thold: {thold}')
    print(sample[f'target_{thold}'].isna().sum())
    sample[f'target_{thold}'] = sample[f'target_{thold}'].fillna(sample[f'target_new_{thold}'])
    print(sample[f'target_{thold}'].isna().sum())
    print()

In [ ]:
for thold in reversed(tholds):
    print(f'curr thold: {thold}')
    print(sample['target_0.55'].isna().sum())
    sample['target_0.55'] = sample['target_0.55'].fillna(sample[f'target_{thold}'])
    print(sample['target_0.55'].isna().sum())
    print()
    
sample['target_0.55'] = sample['target_0.55'].fillna(39)

In [ ]:
preds = pd.DataFrame({'index': sample['index'], 'target': sample['target_0.55']})

In [ ]:
preds.to_csv(f'1epoch_gemma_bf16_thold_0.55_comb_new_fillna39_step_0.0125_from_10_to_55.csv', index=False)

In [ ]:
# 1epoch_gemma_bf16_thold_0.55_comb_new_fillna39_step_0.0125_from_10_to_55.csv BEST